# Homework 2: Spectral analysis

In this notebook we explore the convective and wave motions in terms of their spatial and temporal spectra.  In the final step we will combine spectral and statistical methods and compare how useful they are with regard to characterizing the different types of motions and distinguish between them.

We will again
* Load the 3D data for a given dump
* Extract the 3D velocity components in an already interpolated form on a sphere of given radius

We then perform:
* A Fourier analysis of the spherically averaged $u_\mathrm{r}$ magnitude at a given radius 
* A spectral decomposition of the $u_\mathrm{r}$ distribution on a sphere at a given radius 


Familiarize yourself with this template notebook. In the first part the temporal spectrum analysis is demonstrated, and in the second part the spatial analysis is demonstrated. Then use this template to explore the following questions. 

#### Questions
1. What is the main difference between the $u_r$ spatial spectra for convective and wave regions?
    * Quantify the key difference in the following way:
        - take the asymetric moving average of the power
        - find the $l$ for which the power is maximum
    * Implement this _metric_ as a function that can be added into a data analysis pipeline in a python script.
2. Create temporal spectra for different radii, including in the wave region, in the convection region and in the transition region. 
    * How does the length of the time series impact the spectrum? You can check that by modifying the `dump_max` variable in the demo code below.
    * How does the time step size impact the spectrum? You can vary that simply by choosing a larger value for the `step` variable in the demo code below.
    * How does the temporal spectrum differ between the convection and wave region? 
    * Can you find a _metric_ also in this case that would reduce the difference between temporal spectra of the wave region and the convection region? (I could not, so maybe you don't want to try too hard. It is not needed for the assignment.) 

In [ ]:
%pylab ipympl
import os, sys, time
from multiprocessing import Pool
from scipy import stats
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from nugridpy import utils as ut

# libraries for spherical harmonics spectra
import pyshtools.expand        
import pyshtools.spectralanalysis

# for use on niagara jupyter 
#ppmpy_dir = '/scratch/f/fherwig/fherwig/repos/PyPPM'
# for use on ppmstar hub on https://www.ppmstar.org
ppmpy_dir = "/user/repos/PyPPM"

sys.path.insert(0,ppmpy_dir)
from ppmpy import ppm
print(inspect.getfile(ppm))

# set cycling combination of color-blind labels, glyphs, styles
lll= 2*['-', '--', ':', '-.']
markers = ['X','h','<','>','s','^','d','X','p']
random.shuffle(lll)
cb = ut.linestylecb # colours
CB_color_cycle = [cb(i)[2] for i in range(8)]
rc('axes', prop_cycle=(cycler('color', CB_color_cycle[0:8]) + cycler('marker',markers[0:8])+cycler('linestyle',lll)))
rc('axes', prop_cycle=(cycler('color', CB_color_cycle[0:8]) +cycler('linestyle',lll)))

# named tuple for using rprofs and momsdata
# moms data are almost always used alongside rprof data and therefore it is recommended 
# to create a convenient dictionary that will hold the rprof and moms instance 
import collections
hydro = collections.namedtuple('hydro', ['moms','rprof'])

# turn off matplotlib messages
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

def fig_init(ifig=1):
    close(ifig);figure(ifig)

In [ ]:
# there are two data access points
# the first is slow but has all dumps, loading one dump takes about 20min
data_dir = '/user/niagara.scratch.ppmstar'
# the second is faster, but has only these dumps:
# 4120  4183  4325  4447  4474  4550
# use this as a default
data_dir = '/data/ASDR-team/PPMstar/H-core-M25/'
run_dir = 'M276-1000x-768'

moms_dir = os.path.join(data_dir,run_dir,'moms/myavsbq')
rprof_dir = os.path.join(data_dir,run_dir,'prfs')

# M276 - a list of variables available in these moms files:
var_list = ['xc','ux','uy','uz','|ut|','|ur|','|w|','T','rho','fv'] 
dump = 4447    # initialize with this dump

In [ ]:
# initialize a collection of an instance of a radial profile and 3D briquette data set (called "moms" data)
start_time = time.time()
myrun = hydro(ppm.MomsDataSet(moms_dir,init_dump_read=dump,dumps_in_mem=2,var_list=var_list,\
                              rprofset=ppm.RprofSet(rprof_dir),verbose=0),ppm.RprofSet(rprof_dir))
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

In [ ]:
rph = myrun.rprof.get_history()
# rph.get_variables()

In [ ]:
# make sure that the dumps are at equidistant times so that the DFT routines will work
# we can only do the DFT in the second part, with smaller time steps
fig_init(302)
plot(rph.get('NDump'),rph.get('time(mins)'))
mod_minmax_dft = (3060,6560)

In [ ]:
# check radial profiles visually for any available radial profile quantity
myrun.rprof.rprofgui(ifig=1,title=run_dir)

## Temporal spectrum

Try the following analysis for a radius corresponding to the wave region (e.g. 2100) and to the convection region (e.g.1000). Are you able to find a model function that would allow to quantify the presence of a power maximum around $\nu = 2.5 \mathrm{\mu Hz}$?

In [ ]:
# see all variables available in profile data
# rp =  myrun.rprof.get_dump(4000)
# rp.get_lr_variables()

In [ ]:
myradius=2100
radius = myrun.rprof.get('R',fname=4000)
# radius

In [ ]:
# find index of radial grid point closest to requested radius
ind = ppm.where_near(myradius,radius)
radius[ind]

In [ ]:
# extract variable at requested radius for all dumps in dump range
# to get time evolution data
times = []
things = []
thing_name = '|Ut|'
thing_name = 'lum1'

dump_min,dump_max = mod_minmax_dft
# dump_max = dump_min + (dump_max-dump_min)//4
step = 1
radius = myrun.rprof.get('R',dump_min)
ind = ppm.where_near(myradius,radius)
for dump in range(dump_min,dump_max+1,step):
    thing = myrun.rprof.get(thing_name,dump)
    things.append(thing[ind])
    time = myrun.rprof.get("t", fname=dump)
    times.append(time)
if thing_name == '|Ut|':
    things=array(things)*1000 # make velocities in km/s
times = array(times)-times[0]

In [ ]:
# detrend
# the luminosity data has a long-term trend due to thermal adjustment that
# needs to me removed, with a 2nd-orer poly

# Fit a first degree polynomial (straight line)
# polyfit returns highest degree coefficient first
a, b, c = polyfit(times, things, 2)

print(f"Coefficients a: {a}  b: {b}  c: {c}")

# check data
fig_init(23)
trend = a*times**2+b*times+c
things_detrend = things/trend
plot(times,things_detrend,'-')


In [ ]:
# Compute the DFT
dft = np.fft.fft(things_detrend)

# Compute the frequencies
N = len(times)
time_spacing = times[1] - times[0]
frequencies = np.fft.fftfreq(N, time_spacing)

# Keep only positive frequencies
positive_frequencies = frequencies[:N // 2]
positive_dft = dft[:N // 2]

# Calculate the power spectrum
power_spectrum = np.abs(positive_dft)**2

In [ ]:
fig_init(2201)
semilogy(positive_frequencies*1.e6, power_spectrum,'-')
xlabel('Frequency ($\mu$Hz)')
ylabel('Power Spectrum')

In [ ]:
# try to find a model function that when fitted reveals the location of
# a possible maximum around 2.5muHz in the wave case 
# I was not able to find such function within reasonable time. Are you?

# Define the model function
def model_function(x, a0, a1, a2, b, c):
    return (a0 + a1 * x + a2 * x**2) * c*np.exp(-b * x)

log_positive_frequencies = log10(positive_frequencies[1:]*1.e6)
log_power_spectrum       = log10(power_spectrum[1:])

# ind_max = where(log_power_spectrum==max(log_power_spectrum))[0][0]
# log_positive_frequencies = log_positive_frequencies[ind_max:]
# log_power_spectrum       = log_power_spectrum[ind_max:]

coefs = polyfit(log_positive_frequencies, log_power_spectrum, 8)
print("Coefficients:",coefs)

p = poly1d(coefs)

# Plotting the data and the fit
fig_init(2202)
scatter( log_positive_frequencies, log_power_spectrum, label='Data')
xx = linspace(log_positive_frequencies[0],log_positive_frequencies[-1],100)
plot(xx,p(xx), label='Fitted function', color='red')
xlabel('$\\log \\nu \\mathrm{(\\mu Hz)}$')
ylabel('power')
legend()
show()

# Print the optimized parameters
print(f"Optimized parameters: a = {a:.3f}, b = {b:.3f}")



## Spectral decomposition in Spherical Harmonics

We will be using the Python package [pyshtools](https://shtools.github.io/SHTOOLS/index.html).

In [ ]:
# check documentation 
# pyshtools.expand.SHExpandDH?

In [ ]:
# pyshtools.spectralanalysis.spectrum?

### Expansion in spherical harmonics

In [ ]:
# store ell and power for different radii in dictionary
spatial_specs = {}

In [ ]:
# u_r analysis plots
myradius=1200
var = '|ur|'

# max npoints
lmax_r, N, npoints = myrun.moms.sphericalHarmonics_lmax(myradius)
print(lmax_r, N, npoints)

In [ ]:
# Calculate spherical harmonics up to lmax
var_interp = myrun.moms.sphericalHarmonics_format(var, myradius, lmax=lmax_r)
# get coefficients and power
coeffs = pyshtools.expand.SHExpandDH(var_interp, sampling=2)
power_ell = pyshtools.spectralanalysis.spectrum(coeffs, unit='per_l')
ell = np.arange(0, lmax_r+1)
spatial_specs[myradius] = [ell[1:],1.e12*power_ell[1:]] # exclude l=0

In [ ]:
myradius=1200
fig_init(myradius)

loglog(spatial_specs[myradius][0],spatial_specs[myradius][1],'-')
ell_scale = 3
xx = np.linspace(0.6*ell_scale,0.7*ell[-1],5)
fac = 1./(ell_scale**(-5/3))
pow_scale = float(spatial_specs[myradius][1][spatial_specs[myradius][0] == ell_scale])
pmax = spatial_specs[myradius][1].max()
loglog(xx,pow_scale*fac*xx**(-5/3),'--',lw=0.5,)
ylim(3*spatial_specs[myradius][1][-1],3*pmax)
ylabel(' power / $\mathrm{[m^2/s^2]}$'); xlabel('$l$')
text(1.2*xx[0],0.2*1.e12*pow_scale*fac*xx[1]**(-5/3),'$l^{-5/3}$')

# add here the code to find the max of the smoothed power


## Create script version
Create a function that can go into a python mpi script: